In [91]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [92]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot = True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [93]:
n_input = 28 * 28
n_noise = 128

# 은닉층 노드 수
n_hidden = 256

total_epoch = 100
batch_size = 100
learning_rate = 0.0002 # 학습률

In [94]:
X = tf.placeholder(tf.float32, [None, n_input])
# 생성망에 들어갈 입력데이터(정규 분포를 따르는 128개의 데이터)
Z = tf.placeholder(tf.float32, [None, n_noise])

## 생성자 신경망
- W (가중치)
- b (편향)

In [95]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

## 판별기 신경망

In [96]:
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))

D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev = 0.01))
D_b2 = tf.Variable(tf.zeros([1]))
# 최종 0부터 1 사이의 값 하나

## 생성자(G) 신경망 구성
- 무작위 노이즈를 받아 가중치와 편향을 반영한 은닉층을 구성한다.
- Activation 함수를 이용해 최종 결과값 0~1 사이 반환

#### 노이즈 데이터 발생을 위한 노이즈 생성 함수
- def get_noise(데이터 개수, 노이즈 개수)

In [97]:
# 노이즈 생성 함수
def get_noise(batch_size, n_noise) :
    return np.random.normal(size=(batch_size, n_noise))

In [98]:
# 생성자 신경망 직접 구성
def generator(noise_z) :
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    
    return output

## 판별자 신경망 구성
- 구분자 신경망 구성, 가중치와 편향을 반영한 데이터 출력
- Activation 함수를 이용해 최종 결과 반환

In [99]:
def discriminator(inputs) :
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    
    return output

In [100]:
# 노이즈를 이용, 랜덤 이미지 생성
# Z는 실행시 noise가 입력됨
G = generator(Z)

# 노이즈를 이용하여 생성한 이미지가 진짜 이미지인지 판별
D_fake = discriminator(G)

# 진짜 이미지를 이용해 판별한 값
D_real = discriminator(X)

## 비용함수, 최적화 함수

In [101]:
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_fake))

loss_G = tf.reduce_mean(tf.log(D_fake))

In [102]:
# loss_D 를 구할 때는 판별기 신경망에 사용되는 변수만 사용하고,
# loss_G 를 구할 때는 생성기 신경망에 사용되는 변수만 사용하여 최적화
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [103]:
### 오차의 최소화가 아닌
### loss_D와 loss_G의 최대화가 목표

In [104]:
# 최적화 대상인 loss_D와 loss_G에 음수 부호
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,
                                                        var_list = D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,
                                                        var_list = G_var_list)

In [107]:
!mkdir samples

## 모델 학습

In [105]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

In [108]:
for epoch in range(total_epoch) :
    for i in range(total_batch) :
        batch_xs, batch_ys = mnist.train.next_batch(batch_size) # 판별망의 데이터
        noise = get_noise(batch_size, n_noise) # 생성망의 입력 데이터
        
        # 판별기와 생성기 신경망 각각 학습
        _, loss_val_D = sess.run([train_D, loss_D],
                                 feed_dict = {X: batch_xs, Z : noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                 feed_dict = {Z: noise})
        
    print("Epoch : {}, 판별망 성능 : {:.4f}, 생성망 성능 : {:.4f}".format
         (epoch, loss_val_D, loss_val_G))
    
    if epoch == 0 or (epoch+1) % 10 == 0 :
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict = {Z: noise})
        
        fig, ax = plt.subplots(1, sample_size, figsize = (sample_size,1))
        
        for i in range(sample_size) :
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28,28)))
            
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
        
print('FIN')

Epoch : 0, 판별망 성능 : -0.9344, 생성망 성능 : -1.8741
Epoch : 1, 판별망 성능 : -0.9661, 생성망 성능 : -1.6740
Epoch : 2, 판별망 성능 : -0.7129, 생성망 성능 : -2.0492
Epoch : 3, 판별망 성능 : -0.8009, 생성망 성능 : -1.7945
Epoch : 4, 판별망 성능 : -0.8022, 생성망 성능 : -2.0621
Epoch : 5, 판별망 성능 : -0.8367, 생성망 성능 : -1.8170
Epoch : 6, 판별망 성능 : -0.7007, 생성망 성능 : -1.9916
Epoch : 7, 판별망 성능 : -0.7566, 생성망 성능 : -2.1235
Epoch : 8, 판별망 성능 : -0.6544, 생성망 성능 : -2.0534
Epoch : 9, 판별망 성능 : -0.7952, 생성망 성능 : -2.0358
Epoch : 10, 판별망 성능 : -0.7468, 생성망 성능 : -2.2535
Epoch : 11, 판별망 성능 : -0.6521, 생성망 성능 : -1.9900
Epoch : 12, 판별망 성능 : -0.6729, 생성망 성능 : -2.0487
Epoch : 13, 판별망 성능 : -0.7142, 생성망 성능 : -2.0637
Epoch : 14, 판별망 성능 : -0.7006, 생성망 성능 : -1.8946
Epoch : 15, 판별망 성능 : -0.5934, 생성망 성능 : -2.1037
Epoch : 16, 판별망 성능 : -0.7893, 생성망 성능 : -1.9735
Epoch : 17, 판별망 성능 : -0.5803, 생성망 성능 : -1.9748
Epoch : 18, 판별망 성능 : -0.6999, 생성망 성능 : -1.8738
Epoch : 19, 판별망 성능 : -0.6275, 생성망 성능 : -1.8869
Epoch : 20, 판별망 성능 : -0.6357, 생성망 성능 : -2.1555
Epoch : 21, 판별망 성능 : -0